In [ ]:
import harp
import numpy as np
from pathlib import Path
import os
import utils
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import signal
import copy
from datetime import datetime, timedelta

## Loading

In [ ]:
# data_path = Path('/home/ikharitonov/RANCZLAB-NAS/data/ONIX/20240730_Mismatch_Experiment/MMclosed&Regular_130824/2024-08-13T12-53-01_B3M3')
# photometry_path = Path('/home/ikharitonov/RANCZLAB-NAS/data/ONIX/20240730_Mismatch_Experiment/MMclosed&Regular_130824/photometry/B3M3_MMclosed&Regular_day2/2024_08_13-14_57_35')
data_path = Path('/home/ikharitonov/RANCZLAB-NAS/data/ONIX/20240730_Mismatch_Experiment/GRAB_MMclosed&Regular_230824/2024-08-23T12-51-05_B3M7')
photometry_path = Path('/home/ikharitonov/RANCZLAB-NAS/data/ONIX/20240730_Mismatch_Experiment/GRAB_MMclosed&Regular_230824/photometry/B3M7_MMclosed&Regular_day2/2024_08_23-14_54_36')

In [ ]:
streams = utils.load_registers(data_path)
print(streams[0].keys())
print(streams[1].keys())

## Unit conversion for running

In [ ]:
resolution = 12000 # counts per inch
inches_per_count = 1 / resolution
meters_per_count = 0.0254 * inches_per_count
print(meters_per_count)

dt = 0.01 # for OpticalTrackingRead0Y(46)
linear_velocity = meters_per_count / dt # meters per second per count
print(linear_velocity)

In [ ]:
streams[0]['OpticalTrackingRead0X(46)'] * linear_velocity * 100

In [ ]:
plt.plot(streams[0]['OpticalTrackingRead0X(46)'] * linear_velocity * 100)
plt.xlabel('time (seconds)')
plt.ylabel('running speed (cm/s)')
plt.show()

## Synchronising photometry signal

In [ ]:
Fluorescence = pd.read_csv(photometry_path/'Fluorescence.csv', skiprows=1, index_col=False)
Fluorescence = Fluorescence.drop(columns='Unnamed: 5')
Fluorescence

In [ ]:
# selecting rows of Fluorescence.csv where the Events column is not equal to NaN - where events have occurred
Fluorescence[Fluorescence['Events'].notna()]

In [ ]:
Fluorescence[Fluorescence['Events'].notna()]['Events'].unique()

In [ ]:
# loading OnixDigital file which has HARP timestamps ('Seconds' column - seconds elapsed counting from HARP reference epoch, 1904-01-01T00-00-00 )
OnixDigital = utils.read_onix_digital(data_path)
OnixDigital

In [ ]:
# adding a new 'Seconds' column into Fluorescence and populating it with NaNs, then setting matching rows with timestamp values from OnixDigital (each row of OnixDigital corresponds to a row of Fluorescence[Fluorescence['Events'].notna()] )
Fluorescence['Seconds'] = np.nan
Fluorescence

In [ ]:
# Setting the rows of Seconds column where there are events with HARP timestamp values from OnixDigital
Fluorescence.loc[Fluorescence['Events'].notna(), 'Seconds'] = OnixDigital['Seconds'].values

In [ ]:
# estimate the very first and very last values of Seconds column in Fluorescence to be able to interpolate between
first_val_to_insert = Fluorescence[Fluorescence['Events'].notna()].iloc[0]['Seconds'] - Fluorescence[Fluorescence['Events'].notna()].iloc[0]['TimeStamp'] / 1000
# first_val_to_insert = Seconds value of the first Event to occur - seconds elapsed since start of recording (converted from ms)
last_val_to_insert = Fluorescence[Fluorescence['Events'].notna()].iloc[-1]['Seconds'] + (Fluorescence.iloc[-1]['TimeStamp'] / 1000 - Fluorescence[Fluorescence['Events'].notna()].iloc[-1]['TimeStamp'] / 1000)
# last_val_to_insert = Seconds value of the last Event to occur + seconds elapsed between the last row of Fluorescence and the last event to occur
print(first_val_to_insert)
print(last_val_to_insert)

In [ ]:
Fluorescence.loc[0, 'Seconds'] = first_val_to_insert
Fluorescence.loc[-1, 'Seconds'] = last_val_to_insert
print(Fluorescence.iloc[0]['Seconds'])
print(Fluorescence.iloc[-1]['Seconds'])

In [ ]:
# Applying default Pandas interpolation to Seconds
Fluorescence[['Seconds']] = Fluorescence[['Seconds']].interpolate()

## Plotting together

In [ ]:
ExperimentEvents = utils.read_exp_events(data_path)
ExperimentEvents

In [ ]:
halts_applied_at = ExperimentEvents[ExperimentEvents['Value']=='Apply halt: 1s'].Seconds.values
print(halts_applied_at)

In [ ]:
# A = None
# B = None
A = 439250
B = A + 60

plt.figure(figsize=(12,6))
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(12, 12))

stream46_seconds_elapsed = (streams[0]['OpticalTrackingRead0X(46)'].index - harp.REFERENCE_EPOCH).total_seconds()
ax[0].plot(stream46_seconds_elapsed, streams[0]['OpticalTrackingRead0X(46)'] * linear_velocity * 100)
ax[0].set_title('Running speed recorded with optical tracking sensor')
ax[0].set_xlabel('time (seconds)')
ax[0].set_ylabel('running speed (cm/s)')
for halt in halts_applied_at:
    ax[0].axvline(halt, color='r', alpha=0.2)
ax[0].set_xlim([A, B])

photometry_seconds_elapsed = Fluorescence['Seconds'].values
ax[1].plot(photometry_seconds_elapsed, Fluorescence['CH1-470'].values)
ax[1].set_title('Photometry CH1-410')
ax[1].set_xlabel('time (seconds)')
ax[1].set_ylabel('signal amplitude')
for halt in halts_applied_at:
    ax[1].axvline(halt, color='r', alpha=0.2)
ax[1].set_xlim([A, B])

plt.show()

## Loading photodiode and clock signal

In [ ]:
photo_diode = utils.read_photodiode(data_path)
clock = utils.read_clock(data_path)
print(photo_diode.shape)
print(clock.shape)

In [ ]:
plt.plot(photo_diode[0,0,:])
plt.title('buffer 0, channel 0')
plt.show()

plt.plot(photo_diode[0,1,:])
plt.title('buffer 0, channel 1')
plt.show()

plt.plot(photo_diode[123,0,:])
plt.title('buffer 123, channel 0')
plt.show()

In [ ]:
plt.plot(clock[:200])
plt.show()